In [4]:
# pip install --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.1/vinicius.george@sga.pucminas.br/ABD5-4359-8FC7-5A22-091C-127B-C46B-F481/GraphLab-Create-License.tar.gz

# Análise de sentimentos em Python!

In [1]:
# Primeiramente, vamos importar as bibliotecas que serão utilizadas.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
%matplotlib inline

nltk.download('all-corpora')

# teremos uma base composta por 6 sentimentos (labels): Alegria, Medo, Raiva, Tristeza, Surpresa e Nojo.


[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/jupyterlab/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/jupyterlab/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/jupyterlab/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/jupyterlab/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/jupyterlab/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/jupyterlab/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading pa

True

In [2]:
base_treinamento = [
    ('este trabalho e agradável','alegria'),
    ('gosto de ficar no seu aconchego','alegria'),
    ('fiz a adesão ao curso hoje porque eu gostei','alegria'),
    ('eu sou admirada por muitos','alegria'),
    ('adoro como você e','alegria'),
    ('adoro seu cabelo macio','alegria'),
    ('adoro a cor dos seus olhos','alegria'),
    ('somos tão amáveis um com o outro','alegria'),
    ('sinto uma grande afeição por ele','alegria'),
    ('quero agradar meus filhos','alegria')
]

In [3]:
exemplo_base = pd.DataFrame(base_treinamento)
exemplo_base.columns = ['Frase', 'Sentimento']

In [4]:
print('tamanho base treinamento {}'.format(exemplo_base.shape[0]))
exemplo_base.Sentimento.value_counts()

tamanho base treinamento 10


alegria    10
Name: Sentimento, dtype: int64

In [5]:
print((exemplo_base.Sentimento.value_counts() / exemplo_base.shape[0])*100)

alegria    100.0
Name: Sentimento, dtype: float64


In [6]:
# Temos um conjunto relativamente bem balanceado, com referência ao label (sentimento).

# Vamos gerar uma amostra de 20 frases para ter uma ideia sobre o que iremos trabalhar.

exemplo_base.sample(n=10)

,Frase,Sentimento
1,gosto de ficar no seu aconchego,alegria
4,adoro como você e,alegria
5,adoro seu cabelo macio,alegria
7,somos tão amáveis um com o outro,alegria
3,eu sou admirada por muitos,alegria
6,adoro a cor dos seus olhos,alegria
0,este trabalho e agradável,alegria
2,fiz a adesão ao curso hoje porque eu gostei,alegria
9,quero agradar meus filhos,alegria
8,sinto uma grande afeição por ele,alegria


In [7]:
base_teste = [
    ('não precisei pagar o ingresso','alegria'),
    ('se eu ajeitar tudo fica bem','alegria'),
    ('minha fortuna ultrapassa a sua','alegria'),
    ('sou muito afortunado','alegria'),
    ('e beneficio para todos esta nova medida','alegria'),
    ('ficou lindo','alegria'),
    ('achei esse sapato muito simpático','alegria'),
    ('estou ansiosa pela a sua chegada','alegria'),
    ('congratulações pelo seu aniversário','alegria'),
    ('delicadadamente ele a colocou para dormir','alegria'),
    ('a musica e linda','alegria'),
    ('sem musica eu não vivo','alegria')
]

In [8]:
exemplo_base_teste = pd.DataFrame(base_teste)
exemplo_base_teste.columns = ['Frase', 'Sentimento']
print('Tamanho da base de Teste {}'.format(exemplo_base_teste.shape[0]))
exemplo_base_teste.Sentimento.value_counts()

Tamanho da base de Teste 12


alegria    12
Name: Sentimento, dtype: int64

In [9]:
print((exemplo_base_teste.Sentimento.value_counts() / exemplo_base_teste.shape[0]) * 100)

alegria    100.0
Name: Sentimento, dtype: float64


In [11]:
# Stopwords são palavras comuns que normalmente não contribuem para o significado de uma frase, pelo menos com relação ao propósito da informação e do processamento da linguagem natural.
# São palavras como “The” e “a” ((em inglês) ou “O/A” e “Um/Uma” ((em português).

# Muitos mecanismos de busca filtram estas palavras (stopwords), como forma de economizar espaço em seus índices de pesquisa.

In [10]:
lista_Stop = nltk.corpus.stopwords.words('portuguese')
np.transpose(lista_Stop)

array(['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para',
       'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as',
       'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou',
       'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo',
       'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo',
       'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa',
       'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos',
       'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas',
       'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus',
       'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos',
       'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele',
       'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está',
       'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram',
       'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos

In [13]:
# Essas são as StopWords que já são contempladas na Biblioteca do NLTK.

# Se o conjunto de palavras contido no Corpus do StopWords não lhe atender, você pode facilmente adicionar mais palavras que fazem sentido para seu modelo e/ou até mesmo criar sua própria lista de StopWords customizada.

In [11]:
# Inclusão de algumas StopWords para exemplificar a facilidade de trabalhar com NLTK
lista_Stop.append('tipo')
lista_Stop.append('tão')
lista_Stop.append('tudo')
lista_Stop.append('vai')

In [12]:
# Agora podemos criar nossa função para retirar as StopWords do nosso Corpus

def removeStopWords(texto):
    frases = []
    for (palavras, sentimento) in texto:
        # Criamos um list compreheension para extrair apenas as palavras que não estão na Lista_Stop
        semStop = [ p for p in palavras.split() if p not in lista_Stop]
        # Inserindo as frases com os Labels (sentimento) já tratadas pela Lista_Stop
        frases.append((semStop, sentimento))
    return frases

In [ ]:
# Stemming

# Stemming é a técnica de remover sufixos e prefixos de uma palavra, chamada stem.

# Por exemplo, o stem da palavra cooking é cook. Um bom algoritmo sabe que “ing” é um sufixo e pode ser removido.

# Stemming é muito usado em mecanismos de buscas para indexação de palavras.

# Ao invés de armazenar todas as formas de uma palavras, um mecamismo de busca armazena apenas o stem da palavra, reduzindo o tamanho do índice e aumentando a performance do processo de busca.

In [13]:
import nltk
nltk.download('rslp')

def aplica_Stemmer(texto):
    stemmer = nltk.stem.RSLPStemmer()
    # Escolhido o RSLPS pois é especifico da lingua portuguesa
    frases_sem_Stemming = []
    for (palavras, sentimento) in texto:
        com_Stemming = [str(stemmer.stem(p)) for p in palavras.split() if p not in lista_Stop]
        frases_sem_Stemming.append((com_Stemming, sentimento))
    return frases_sem_Stemming

[nltk_data] Downloading package rslp to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [14]:
frases_com_Stem_treinamento = aplica_Stemmer(base_treinamento)
pd.DataFrame(frases_com_Stem_treinamento, columns=['Frase', 'Sentimento']).sample(10)

,Frase,Sentimento
5,"[ador, cabel, maci]",alegria
9,"[quer, agrad, filh]",alegria
7,"[am, outr]",alegria
0,"[trabalh, agrad]",alegria
3,"[admir, muit]",alegria
8,"[sint, grand, afe]",alegria
6,"[ador, cor, olh]",alegria
2,"[fiz, ades, curs, hoj, porqu, gost]",alegria
4,[ador],alegria
1,"[gost, fic, aconcheg]",alegria


In [15]:
frases_com_Stem_teste = aplica_Stemmer(base_teste)

In [16]:
def busca_Palavras(frases):
    todas_Palavras = []
    for (palavras, sentimento) in frases:
        todas_Palavras.extend(palavras)
    return todas_Palavras

In [17]:
palavras_treinamento = busca_Palavras(frases_com_Stem_treinamento)
palavras_teste = busca_Palavras(frases_com_Stem_teste)

In [18]:
print("Quantidade de palavras na base de treinamento {}".format(pd.DataFrame(palavras_treinamento).count()))

Quantidade de palavras na base de treinamento 0    28
dtype: int64


In [19]:
def busca_frequencia(palavras):
    palavras = nltk.FreqDist(palavras)
    return palavras

In [20]:
frequencia_treinamento = busca_frequencia(palavras_treinamento)

In [3]:
# A função most_common do NLTK possibilita visualizar quais as palavras que ocorrem com maior frequência em nosso texto.
# Abaixo listamos apenas as 20 mais comuns. 
# Esta atividade pode nos ajudar a voltar a lista de StopWords e inserir novas palavras para serem excluidas de nossa análise…
# O trabalho do cientista de dados é iterativo, sempre buscamos descobrir tendências, formas de melhorar o modelo, etc…

In [21]:
frequencia_treinamento.most_common(20)

[('ador', 3),
 ('agrad', 2),
 ('gost', 2),
 ('trabalh', 1),
 ('fic', 1),
 ('aconcheg', 1),
 ('fiz', 1),
 ('ades', 1),
 ('curs', 1),
 ('hoj', 1),
 ('porqu', 1),
 ('admir', 1),
 ('muit', 1),
 ('cabel', 1),
 ('maci', 1),
 ('cor', 1),
 ('olh', 1),
 ('am', 1),
 ('outr', 1),
 ('sint', 1)]

In [22]:
# Executamos também para a base de treinamento
frequencia_teste = busca_frequencia(palavras_teste)

In [23]:
#Função para retornar somente as palavras únicas

def busca_palavras_unicas(frequencia):
    freq = frequencia.keys()
    return freq

palavras_unicas_treinamento = busca_palavras_unicas(frequencia_treinamento)
palavras_unicas_teste = busca_palavras_unicas(frequencia_teste)

In [24]:
# Precisamos agora criar uma função para identificar quais palavras únicas estão no documento passado para a função!

def extrator_palavras(documento):
    # Utilizado set() para associar a variavel doc com o parâmetro que esta chegando
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_treinamento:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [25]:
# Devido a necessidade de aplicação da função Extrator_Palavras sobre as bases de Treinamento e Teste, como a variável palavras_unicas_teste exige a aplicação isolada, precisamos criar uma função apartada somente para a base de teste!

def extrator_palavras_teste(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_teste:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [26]:
# Função apply features do NLTK faz o preenchimento se tem ou não (True / False) a característica de acordo do com o parâmetro!

base_completa_treinamento = nltk.classify.apply_features(extrator_palavras, frases_com_Stem_treinamento)
base_completa_teste = nltk.classify.apply_features(extrator_palavras_teste, frases_com_Stem_teste)

In [27]:
# O algortimo NaiveBayes monta a tabela de probabilidade!

classificador = nltk.NaiveBayesClassifier.train(base_completa_treinamento)

In [28]:
print(classificador.labels())

['alegria']


In [29]:
print(classificador.show_most_informative_features())

Most Informative Features
None


In [30]:
# Agora vamos verificar a acurácia do modelo!

print(nltk.classify.accuracy(classificador, base_completa_teste))

1.0


In [31]:
# Para ter uma visão mais clara dos erros e sentimentos que estão em conflito no nosso modelo, vamos gerar uma matriz de confusão do NLTK.

erros = []
for (frase, classe) in base_completa_teste:
    #print(frase)
    #print(classe)
    resultado = classificador.classify(frase)
    if resultado != classe:
        erros.append((classe, resultado, frase))

In [32]:
from nltk.metrics import ConfusionMatrix
esperado = []
previsto = []
for (frase, classe) in base_completa_teste:
    resultado = classificador.classify(frase)
    previsto.append(resultado)
    esperado.append(classe)

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

        |  a |
        |  l |
        |  e |
        |  g |
        |  r |
        |  i |
        |  a |
--------+----+
alegria |<12>|
--------+----+
(row = reference; col = test)



In [35]:
# Para realmente saber como nosso modelo esta atuando na análise de sentimento, vamos incluir algumas frases aleatórias e verificar qual tag ele marca nossa frase!
# teste = ‘Nossa, que notícia maravilhosa!’

teste = 'Nossa, que notícia maravilhosa!'
testeStemming = []
stemmer = nltk.stem.RSLPStemmer()
for (palavras_treinamento) in teste.split():
    comStem = [p for p in  palavras_treinamento.split()]
    testeStemming.append(str(stemmer.stem(comStem[0])))

novo = extrator_palavras(testeStemming)

#print(classificador.classify(novo))
distribuicao = classificador.prob_classify(novo)
for classe in distribuicao.samples():
    print('%s: %f' % (classe, distribuicao.prob(classe)))

alegria: 1.000000


In [37]:
# Nosso modelo se comportou corretamente, identificou a frase como “Surpresa”, com 42%, bem acima dos demais sentimentos!!!
# Vamos testar mais uma frase!
# teste = ‘Pqp, que trânsito chato da porra!’

teste = 'Pqp, que trânsito chato da porra!'
testeStemming = []
stemmer = nltk.stem.RSLPStemmer()
for (palavras_treinamento) in teste.split():
    comStem = [p for p in  palavras_treinamento.split()]
    testeStemming.append(str(stemmer.stem(comStem[0])))

novo = extrator_palavras(testeStemming)

#print(classificador.classify(novo))
distribuicao = classificador.prob_classify(novo)
for classe in distribuicao.samples():
    print('%s: %f' % (classe, distribuicao.prob(classe)))

alegria: 1.000000


In [ ]:
# Neste trabalho foi utilizada uma base de dados pequena (646 frases em treino e 256 frases em teste).

# A análise de sentimento pode ser utilizada em redes sociais para identificar o que os clientes estão falando de uma determinada marca, assunto, etc… Bastaria conectar-se a uma API, buscar as informações de forma online e realizar a análise de sentimento simultaneamente.